# GRÁFICOS

In [1]:
# %%
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.io as pio
import re

pio.renderers.default = "notebook_connected"

DATABASE_NAME = "internet_governance_news.db"

def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("SELECT * FROM articles", conn)
    conn.close()
    return df

df_all = load_articles()
print(f"📦 Total de registros no banco: {len(df_all)}")
df_all.head()


📦 Total de registros no banco: 3523


,id,title,date,author,url,source
0,610,"50 milhões de brasileiros já usam IA, mas pote...",09 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/50-milhoes-de-...,CGI Notícias
1,611,15ª Semana de Infraestrutura da Internet debat...,09 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/15-semana-de-i...,CGI Notícias
2,612,NIC.br celebra 20 anos de atuação pelo desenvo...,05 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/nic-br-celebra...,CGI Notícias
3,613,No Dia Internacional da Pessoa com Deficiência...,03 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/no-dia-interna...,CGI Notícias
4,614,Estudo do Cetic.br revela demanda por formação...,2025-11-25,CGI.br Notícias,https://cgi.br/noticia/releases/estudo-do-ceti...,CGI Notícias


### Preparo das datas

In [2]:
# %%
df_time = df_all.copy()

df_time["date_clean"] = pd.to_datetime(
    df_time["date"].str.extract(r"(\d{2}/\d{2}/\d{4})")[0],
    dayfirst=True,
    errors="coerce"
)

df_time = df_time.dropna(subset=["date_clean"])

print(f"⏱️ Registros com data válida: {len(df_time)}")


⏱️ Registros com data válida: 1279


### Contagem de notícias por fonte (6 SITES)

In [3]:
df_all["source"].value_counts()

source
European Commission     899
CGI Notícias            836
Câmara dos Deputados    778
NIC Notícias            499
Senado Federal          329
Agência Senado          172
CGI Reuniões             10
Name: count, dtype: int64

### Gráfico de Pizza (TODAS as fontes)

In [10]:

source_counts = df_all["source"].value_counts().reset_index()
source_counts.columns = ["source", "count"]

px.pie(
    source_counts,
    names="source",
    values="count",
    title="Distribuição de Notícias por Fonte (Todos os Sites)"
).show()


### Total de notícias ao longo do tempo

In [5]:

timeline = (
    df_time["date_clean"]
    .value_counts()
    .sort_index()
    .reset_index()
)

timeline.columns = ["data", "quantidade"]

px.line(
    timeline,
    x="data",
    y="quantidade",
    title="Total de Notícias ao Longo do Tempo"
).show()


### Top 15 notícias mais recentes

In [6]:
# %%
top15 = (
    df_time.sort_values("date_clean", ascending=False)
           .head(15)
           .copy()
)

top15["ordem"] = range(1, len(top15) + 1)

px.bar(
    top15,
    x="ordem",
    y="title",
    orientation="h",
    title="Top 15 Notícias Mais Recentes"
).show()


### Nuvem de palavras (títulos – TODOS os sites)

In [8]:
# %%
text = " ".join(df_all["title"].astype(str)).lower()
words = re.findall(r"\b\w{4,}\b", text)

word_freq = (
    pd.Series(words)
    .value_counts()
    .head(20)
    .reset_index()
)

word_freq.columns = ["palavra", "frequencia"]

px.treemap(
    word_freq,
    path=["palavra"],
    values="frequencia",
    title="Palavras Mais Frequentes nos Títulos (Todos os Sites)"
).show()


### Comparação direta entre fontes específicas

In [9]:
# %%
compare = (
    df_all["source"]
    .value_counts()
    .reset_index()
)

compare.columns = ["fonte", "quantidade"]

px.bar(
    compare,
    x="fonte",
    y="quantidade",
    title="Comparação de Volume de Notícias por Fonte"
).show()
